In [1]:
from sklearn.utils import shuffle
from tqdm.notebook import tqdm
import tensorflow as tf
import pandas as pd
import numpy as np
from modules.dataset import Directory
from modules.utils import serialize_fold

2022-10-25 08:25:12.805431: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
INPUT_PATH = "dataset/ChestXray NIH"
STRATEGY = tf.distribute.get_strategy()    
BATCH_SIZE = 16
IMG_SIZE = 224
SEED = 42

In [6]:
df = pd.read_csv(f"{INPUT_PATH}/preprocessed_data.csv", index_col=0)
df = df.astype("int16")
df = shuffle(df, random_state=SEED)
df1, df2, df3, df4, df5 = np.split(df, 5)

In [7]:
df

,No Finding,Atelectasis,Consolidation,Infiltration,Pneumothorax,Edema,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia
dataset/ChestXray NIH/images_010/images/00022260_003.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dataset/ChestXray NIH/images_006/images/00012048_007.png,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
dataset/ChestXray NIH/images_007/images/00014352_005.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dataset/ChestXray NIH/images_003/images/00006040_001.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dataset/ChestXray NIH/images_005/images/00011202_002.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dataset/ChestXray NIH/images_009/images/00018865_040.png,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
dataset/ChestXray NIH/images_012/images/00030028_001.png,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
dataset/ChestXray NIH/images_011/images/00027679_000.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dataset/ChestXray NIH/images_001/images/00000211_014.png,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [16]:
pd.concat([df1, df2, df3, df4, df5])

,No Finding,Atelectasis,Consolidation,Infiltration,Pneumothorax,Edema,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia
dataset/ChestXray NIH/images_010/images/00022260_003.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dataset/ChestXray NIH/images_006/images/00012048_007.png,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
dataset/ChestXray NIH/images_007/images/00014352_005.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dataset/ChestXray NIH/images_003/images/00006040_001.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dataset/ChestXray NIH/images_005/images/00011202_002.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dataset/ChestXray NIH/images_009/images/00018865_040.png,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
dataset/ChestXray NIH/images_012/images/00030028_001.png,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
dataset/ChestXray NIH/images_011/images/00027679_000.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dataset/ChestXray NIH/images_001/images/00000211_014.png,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [20]:
NUM_FILES = 112
len(np.array_split(df1, NUM_FILES))

112

In [32]:
sum([df1.shape[0], df2.shape[0], df3.shape[0], df4.shape[0], df5.shape[0]])

112120

In [35]:
sum([np.vstack(np.array_split(eval(f"df{i}"), NUM_FILES)).shape[0] for i in range(1, 5+1)])

112120